In [30]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

all_links = []  #create and empty list
for i in range(5): #extracting the main 5 pages containing all 500 movies
    req = urllib.request.Request('https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page='+str(i), headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    hyperlinks = soup.find_all('a', attrs = {'class':'title'}) 
    all_links.extend(hyperlinks) #adding links to each page as a separate list element.
len(all_links)

500

In the above box we imported all the necessary library to scrape the metacritic website and saved the links to all top
500 movies in a list names hyperlinks.

In [32]:
all_urls = [] 
for i in all_links:
    all_urls.append('https://www.metacritic.com' + i.get('href'))
all_urls    
# appending the hyperlinks of separate movie with names.

['https://www.metacritic.com/movie/the-godfather',
 'https://www.metacritic.com/movie/citizen-kane',
 'https://www.metacritic.com/movie/rear-window',
 'https://www.metacritic.com/movie/casablanca',
 'https://www.metacritic.com/movie/boyhood',
 'https://www.metacritic.com/movie/three-colors-red',
 'https://www.metacritic.com/movie/vertigo',
 'https://www.metacritic.com/movie/notorious-1946',
 'https://www.metacritic.com/movie/singin-in-the-rain',
 'https://www.metacritic.com/movie/city-lights',
 'https://www.metacritic.com/movie/moonlight-2016',
 'https://www.metacritic.com/movie/intolerance',
 'https://www.metacritic.com/movie/pinocchio-1940',
 'https://www.metacritic.com/movie/touch-of-evil',
 'https://www.metacritic.com/movie/seven-samurai',
 'https://www.metacritic.com/movie/au-hasard-balthazar',
 'https://www.metacritic.com/movie/the-lady-vanishes-1938',
 'https://www.metacritic.com/movie/the-treasure-of-the-sierra-madre',
 'https://www.metacritic.com/movie/pans-labyrinth',
 'https

Below we write code to extract movie name, director of the movie and genres of the movie for use. 

In [33]:

director_dict = {} #empty dictionary which will store movie name as key and director names as values
genre_dict = {}   #empty dictionary which will store movie name as key and genre names as values
for i in all_urls:
    req = urllib.request.Request(i, headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    name_div = soup.find('div', attrs = {'class':'product_page_title oswald'}) #getting the exact unique name of a movie
    name = name_div.get_text().strip().replace("\n"," ") # stripping skip line symbol with a space 
    #print(name)
    director_div = soup.find('div', attrs = {'class':'director'}) #getting div where director name is stored
    director_link = director_div.find_all('a') #since director name is also a link we find the hyperlink inide director div
    directors = []     #creating empty list to store name of directors
    for i in director_link:
        directors.append(i.get_text().strip()) # appending director names to the list
    #print(directors)
    genres_div = soup.find('div', attrs = {'class':'genres'}) # doing the same thing we did for directors to genres.
    genres_list = genres_div.find_all('span')
    genres = []
    for i in genres_list:
        genres.append(i.get_text())
    #print(genres[2:])
    director_dict[name] = ", ".join(directors) #since genres name and directors names are stored in a list we 
    genre_dict[name] = ", ".join(genres[2:])   # convert them to strings.
#print(director_dict)
#print(genre_dict)

Now we use pandas library to create a table to store movie name as first column, director name as second and genres name as third.

In [2]:
import pandas as pd
movie_data = pd.DataFrame({'Director':director_dict, 'Genres':genre_dict})

movie_data.index.name = 'Names'
movie_data
movie_data.to_csv('Movies.csv', sep=',', index = True)


NameError: name 'director_dict' is not defined

We create a pandas dataframe from the saved csv file which has movie names, director name, and genres as separate columns.

In [4]:
import pandas as pd #importing pandas library
import csv 
movie_df = pd.read_csv('./Movies.csv', sep=',') #reading values in csv file.
movie_list = []
for i in movie_df.Names:
    movie_list.append(i.lower())
#print(movie_list)

movie_df.head(5)
#print(len(set(movie_df["Names"])))
print(movie_df["Director"][20:30])

20        Alfred Hitchcock
21             Steve James
22          Akira Kurosawa
23    Joseph L. Mankiewicz
24       François Truffaut
25           Sam Peckinpah
26            Jim Sheridan
27              Carol Reed
28         Stanley Kubrick
29          Jasmila Zbanic
Name: Director, dtype: object


Below is the function that takes the name of movie as an input and returns the director of the movie and the genres of the movie.

In [5]:
def movie(): #defining a function
    while True: #to loop until proper input is given
        movie_input = input("Please enter a movie name. Note-> it accepts regex so use [] to nullify special characters ")
        movie = movie_input.lower() #taking input as a movie name and turning to lowercase for verification.
        try: 
            my_filter = movie_df.Names.str.contains(movie, case = False) #only selecting the rows which has the given movie name.
            filtered_movies = movie_df[my_filter] 
            if len(filtered_movies)>0: #executes only if there is such movie in our dataframe.
                directors = list(filtered_movies["Director"]) #putting the directors of movie in a list
                movies = list(filtered_movies["Names"]) #putting the names of movie in a list
                print(movies)
                string1 =  ', '.join(directors) #since directors and genres are on a list we need to split and
                list1 = string1.split(', ')     # join them to convert into individual strings.          
                genres = set(filtered_movies["Genres"])
                string2 =  ', '.join(genres)
                list2 = string2.split(', ')
                print("The directors are "+ ", ".join(set(list1)))
                print("The genres are " + ", ".join(set(list2)))
                break
        except:
            print("Please enter a valid movie name. ")
#movie()


Below is the function that takes the name of director as an input and returns the names of the movie he/she has directed and the genres of the movies directed.

In [6]:
def director(): #defining director function
    while True:
        director_input = input("Please enter a director name. Note-> it accepts regex so use [] to nullify special characters  ")
        director = director_input.lower() # as above taking director as input and turning to lowercase
        try:
            my_filter = movie_df.Director.str.contains(director, case = False) #as defined above only selecting
            filtered_movies = movie_df[my_filter]                              # rows which has the director of input
            if len(filtered_movies)>0:
                movies = list(filtered_movies["Names"])
                genres = list(filtered_movies["Genres"])
                string1 =  ', '.join(genres)
                list1 = string1.split(', ') # converting lists to individual strings
                dict1 = {}
                for i in list1: #to have name of the genre as key and the number of times director has directed a 
                    if i not in dict1: # movie of same genre as value.
                        dict1[i] = 1
                    else:
                        dict1[i] +=1
                print("\n")
                print(f"The movies {director_input} has directed are " +", ".join(movies)+". \n")
                result = " "
                for keys, values in dict1.items(): #displays how many times the director has directed the movie of a 
                    result += f"{keys}:{values}, " #certain genre.
                final = print(f"The most directed genres are {result}")
                return final
        except:
            print("Please enter a valid director name. ")
#director()

Below we write a function that takes two directors as inputs and returns the cosine similarity between the genres
of the movies they have directed. The more they have common genres the higher they have cosine similarity.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity # library for cosine similarity
def cosine(): # defining the cosine function
    while True:
        input1 = input("Please enter the first director name. Note-> it accepts regex so use [] to nullify special characters  ")
        input2 = input("Please enter the second director name. Note-> it accepts regex so use [] to nullify special characters ")
        dir1 = input1.lower()
        dir2 = input2.lower()
        try:
            my_filter1 = movie_df.Director.str.contains(dir1, case = False) 
            filtered_movies1 = movie_df[my_filter1]
            my_filter2 = movie_df.Director.str.contains(dir2, case = False) # so far the process is the same as above 
            filtered_movies2 = movie_df[my_filter2]                        # function just in place of one we take two
            if len(filtered_movies1)>0 and len(filtered_movies2)>0:       # separete inputs of directors
                genres1 = list(filtered_movies1["Genres"])
                string1 =  ', '.join(genres1) # to concatenate all genres in a single string
                list1 = string1.split(', ')   # to de concatenate single strings into separate genres
                genres2 = list(filtered_movies2["Genres"])
                string2 =  ', '.join(genres2) # repeat the above process for second director.
                list2 = string2.split(', ') 
                #print("\n")
                #print(list1)
                #print("\n")
                #print(list2)
                #print("\n")
                common_genres = set(list1).union(set(list2)) # finding which genres have the two directors in common
                
                #c = print(common_genres)
                dict1 = {}
                for i in list1:
                    if i not in dict1:
                        dict1[i] = 1  # this takes the name of genre for director 1 as key and count of genres as value.
                    else:
                        dict1[i] = dict1[i] + 1
                #print("\n")
                #print(dict1)
                dict2 = {}
                for i in list2:   
                    if i not in dict2:
                        dict2[i] = 1  # this takes the name of genre for director 1 as key and count of genres as value.
                    else:
                        dict2[i] = dict2[i] + 1
                #print("\n")
                #print(dict2)
                
                dict_common1 = {}
                for i in common_genres: #we have to make a common dictionary of the genres in order to put the same 
                    dict_common1[i] = 0 # all the genre and if a director hasn't directed a movie in that genre 
                dict_common1.update(dict1) # this simply assigns the value of 0 to that genre key.
                #print("\n")
                #print(dict_common1)
                
                dict_common2 = {}
                for i in common_genres:
                    dict_common2[i] = 0    # doing as above for director 2
                dict_common2.update(dict2)
                #print("\n")
                #print(dict_common2)
                vector1 = [] 
                vector2 = []
                for keys, values in dict_common1.items(): #extract the numerical value for cosine distance calculation
                    vector1.append(values)
                for keys, values in dict_common2.items():
                    vector2.append(values)
                similarity = cosine_similarity([vector1],[vector2]) #calculating the cosine distance
                final = (f"The cosine similarity between {input1} and {input2} is {similarity}")
                return final

        except:
            print("Please enter both valid directors")
        
#print(cosine())    
    

Below we create the main function which asks user what they want to do. find directors and genres of a movie, or the movie and genres a director has directed or cosine similarity of genres between two directors.

In [11]:
def choose():  #defining the function.
    while True: # loop until a proper choice is selected
        choose1 =  input("What do you want to check on metacritic?(Please choose ‘movie’, ‘director’, or ‘comparison.)")
        choice = choose1.lower() # converting to lowercase for easy applicability.
        try:
            if choice == "movie":  # the code executes only one function among the above three according to 
                execute = movie()  # what user prefers among the input choices movie, director and comparison.
            elif choice == "director":
                execute = director()
            elif choice == "comparison":
                execute = cosine()
            return execute
        except:              # goes back and asks user to input properly.
            print("Please type among the three options ")
choose()
    

What do you want to check on metacritic?(Please choose ‘movie’, ‘director’, or ‘comparison.)comparison
Please enter the first director name. Note-> it accepts regex so use [] to nullify special characters  francis
Please enter the second director name. Note-> it accepts regex so use [] to nullify special characters kubrick


'The cosine similarity between francis and kubrick is [[0.78817011]]'